**Código Detección de Bots**

In [15]:
#Install libraries
!pip install emoji
!pip install botometer

#Import packages
from datetime import datetime
import emoji
import numpy as np
import tweepy
import time
from botometer import Botometer
import pandas as pd


#define variables
consumer_key = "xuSrX6LK9gfsWnJ1CTQblfL83"
consumer_secret = "lIyGqVBJygUHHQ4ZoHyAc8INULSr1B4WnuCajei13QINe7U3SH"
access_token = "549909021-ksoN2E4V4yAlhAPa5EUuItuiQhSXyIKFEvCHLqMv"
access_token_secret = 'JplD62xq9UPnlQ0NFekGINJZHO1nEpfmw8Myiyl6PyaXS'

#Twitter API authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

#Botometer authentication
rapidapi_key = 'b03c5389camsh0eee1dc021c58b0p1ce137jsndb0f24ec6e3a'

twitter_app_auth={
'consumer_key' : "xuSrX6LK9gfsWnJ1CTQblfL83",
'consumer_secret' : "lIyGqVBJygUHHQ4ZoHyAc8INULSr1B4WnuCajei13QINe7U3SH",
'access_token' : "549909021-ksoN2E4V4yAlhAPa5EUuItuiQhSXyIKFEvCHLqMv",
'access_token_secret' :"JplD62xq9UPnlQ0NFekGINJZHO1nEpfmw8Myiyl6PyaXS"
}
botometer = Botometer(wait_on_ratelimit=True,mashape_key = rapidapi_key,**twitter_app_auth)
#Define Functions
def yes_or_no(question):
  reply = str(input(question+' (Si/No): ')).lower().strip()
  if reply.lower() == 'si':
    return True
  if reply.lower() == 'no':
    return False
  else:
    return False

def report_user(screen_name):
    #usuario a bloquear
    #screen_name = 'Valeria95111320'
    
    #bloquee el usuario
    perform_block = False
    
    #Reportarlo con la API de Twitter
    api.report_spam(screen_name = screen_name, perform_block=perform_block)
def denunciar_bots(clasificacion_bots):
  reported_users=[]
  for i in range(len(clasificacion_bots)):
    if clasificacion_bots['respuesta'][i] == 'Bot':
      try:
        report_user(clasificacion_bots['users'][i])
        print('reportando a '+str(clasificacion_bots['users'][i]))
        reported_users.append(clasificacion_bots['users'][i])
      except:
        pass
  bd = pd.DataFrame(reported_users,columns = ['bots denunciados'])
  bd.to_excel('/content/bots_denunciados.xlsx',index = False)

def porcentajeRT(data_set):
    #Evaluar si es Retweet o post original.
    data_set['RT']=[1 if 'RT @' in str(x)[:] else 0 for x in data_set.text]
    NumRT = data_set['RT'].sum()#Número de RT en todos los tweets
    NumTweets = len(data_set)#Número total de Tweets
    NumTweetsORI = NumTweets - NumRT #Número de Tweets Originales
    PorcentajeRT = NumRT/NumTweets #Porcentaje de RT sobre Tweets Totales
    return PorcentajeRT
def porcentajeMultimedia(data_set):
    data_set['Multimedia']=[1 if 'https://t.' in str(x)[:] else 0 for x in data_set.text]
    NumMultimedia = data_set['Multimedia'].sum()#Número de RT en todos los tweets
    NumTotalMulti = len(data_set)#Número total de Tweets
    NumTweetsSinMulti = NumTotalMulti - NumMultimedia #Número de Tweets Originales
    PorcentajeMulti = NumMultimedia/NumTotalMulti #Porcentaje de RT sobre Tweets Totales
    return PorcentajeMulti

def conteoTendencias(HashtagList):
    count = 0
    index = []
    lista = ['Petro','Uribe','Kpop','Molano','ParoNacional','Paro']
    matchers=lista +list(tendencias_saboteadas['Tendencias Saboteadas'])
    matching = [True for s in HashtagList if any(xs in s for xs in matchers)]
    for i in range(len(HashtagList)):
        if HashtagList[i].lower() in [x.lower() for x in  list(tendencias_saboteadas['Tendencias Saboteadas'])] or 'kpop' in HashtagList[i].lower():
            #or [True for s in HashtagList[i].lower() if any(xs in s for xs in ['Petro','Uribe','Kpop'])][0]:
            count=count +1
            index.append(i)
        else:
            count = count        
    return [count,index]

def porcentajeTendencia(data_set):
    numTendencias = conteoTendencias(data_set['hashtags'])[0]
    porTend = numTendencias/len(data_set)
    return porTend

def getscore(x):
    try:
        scores = botometer.check_account(x)["display_scores"]["universal"]["overall"]
        return scores
    except:
        return 0

def getbotometer(clasbots): #clasbots en formato DataFrame, Threshold puede ser un valor entre 0 y 100 que será calculado como el porcentaje de umbral con el cual se medirá si es Bot o no lo es
    #threshold: umbral del botometer
    threshold = 45
    
    #username : usuario a evaluar
    #users = clasbots[clasbots['Botometer'] =='Posible Bot']['username']
    
    clasbots['Botometer'] = clasbots['username'].apply(lambda x: getscore(x))    
    #clasbots.to_excel('Bot_clasificacion.xls',index = False)    
    clasbots.to_excel('/Users/mac/Documents/DAPRE/data_bot_clasificacion.xlsx',index = False)

def isbot():

    bot = 0
    if results[0].author.verified == False:
        bot = bot*0
        
        if results[0].author.default_profile_image == True:
            bot= bot+0.08
            
        else: 
            bot = bot

        if results[0].author.default_profile == True:
            bot= bot+0.07

        else: 
            bot = bot

        if (results[0].author.followers_count < 10000) :
            bot= bot+0.10

        else: 
            bot = bot

        if (results[0].author.friends_count < 10000) :
            bot= bot+0.10

        else: 
            bot = bot

        #if (df.Interactions < 200) : #Get Interactions with a function
        #    bot= bot+0.15  

        #else: 
        #    bot = bot

        if results[0].author.location is np.nan:
            bot= bot+0.15

        else: 
            bot = bot

        if (results[0].author.url is np.nan or results[0].author.url == 'None'):
            bot= bot+0.20

        else: 
            bot = bot

        if  results[0].author.description is np.nan or len(results[0].author.description) <50 or not(bool(emoji.get_emoji_regexp().search(results[0].author.description))):
            bot= bot+0.15

        else: 
            bot = bot

        if (datetime.now().year-results[0].author.created_at.year) == 1:
            bot= bot+0.15
        
        else: 
            bot = bot
        print('BOT RESULTADO: '+str(bot))
        return bot
    else:
        bot=bot*1
    print('BOT RESULTADO: '+str(bot))
    return bot
    
if __name__ == "__main__":
  reportar = yes_or_no('¿Quiere reportar a los usuarios clasificados como "BOT"?')
  tendencias_saboteadas = pd.read_excel('/content/Tendencias saboteadas.xlsx')#lista de hashtag saboteados
  #users = ['gustavobolivar','ivanduque','petrogustavo','tavanine','andricio360','valeria95111320']
  #users = autores_paro_nacional['User']
  #users = pd.read_excel('/content/Tweets#ElParoNoMeRepresenta_2021-05-31.xlsx',sheet_name='Hoja1')['user_screen_name']
  listaCuentas = pd.read_excel('/content/ListaCuentas.xlsx')
  users = listaCuentas['Cuenta']
  users_not_found=[]
  porRT=[]
  porMulti=[]
  porTendencias=[]
  porConteo=[]
  porSource=[]
  sumaTotal=[]
  usuarios = []
  boto = []
  for k in range(0, len(users)):
      try:
          api.get_user(users[k])
          print('Conteo Usuarios Reales: '+str(round((k+1)/len(users)*100))+'%')
          if api.get_user(users[k]).protected:
              users_not_found.append(users[k])       
      except:
          print('User Not Found')
          users_not_found.append(users[k])
          
  print("Usuarios no encontrados: "+str(users_not_found))
  users = [s for s in users if s not in users_not_found]
  print("Usuarios a validar: "+str(len(users)))
  respuestaFinal = []

  for i in range(0,len(users)):
      print('Usuario a validar: '+str(users[i]))
      results_Duque_RT = []
      counter = 1
      tweetsPerQry = 100
      screen_name = users[i]
      #Verifica que el usuario existe
      try:
          for tweet in tweepy.Cursor(api.user_timeline, id = screen_name, count=tweetsPerQry, tweet_mode="extended").items(200):                                
              results_Duque_RT.append(tweet)
              counter+=1
              #print(counter)
      except tweepy.error.TweepError:
          time.sleep(920)
          continue
      print('Total Tweets so far %s' %(str(len(results_Duque_RT))))
      
      results = results_Duque_RT
      
      #Creación del DataSet con las variables por tweet
      columnas_data_set = ['username','text','created_at','retweet_count','favorite_count','url_Tweet','source','lang']
      data_set = pd.DataFrame([],columns = columnas_data_set)
      
      data_set['text']=[tweet.full_text for tweet in results]
      data_set['created_at']=[tweet.created_at for tweet in results]
      data_set['retweet_count']=[tweet.retweet_count for tweet in results]
      data_set['favorite_count']=[tweet.favorite_count for tweet in results]
      data_set['url_Tweet']=["https://twitter.com/"+users[i]+"/status/"+tweet.id_str for tweet in results]
      data_set['source']=[tweet.source for tweet in results]
      data_set['lang']=[tweet.lang for tweet in results]
      data_set['username'] = data_set['text'].apply(lambda x: users[i])
      HashtagList=[]
      Usermen=[]
      
      for tweet in results:
          l=""
          for p in tweet.entities["hashtags"]:
              if l=="":
                  l=p["text"]
              else:
                  l=l+","+p["text"]

          HashtagList.append(l)
          mum=""
          fulltext = [tweet.full_text for tweet in results][0]

          for q in tweet.entities["user_mentions"]:
              if mum=="":
                  mum=q["screen_name"]
              else:
                  mum=mum+","+q["screen_name"]

          Usermen.append(mum)



      data_set["hashtags"]=HashtagList
      data_set["user_mentions"]=Usermen    
      #Primer Filtro    
      if len(results)>0:
          primer_filtro = isbot()

          if primer_filtro >0.4:
              
              #Evalua porcentaje de RT sobre Tweets Totales
              var1 = porcentajeRT(data_set)
              porRT.append(var1)
              print('porRT: '+str(var1))

              #detectar multimedia en los tweets
              var2 = porcentajeMultimedia(data_set)
              porMulti.append(var2)
              print('porMulti: '+str(var2))

              #comparar tendencias 
              var3 = porcentajeTendencia(data_set)
              porTendencias.append(var3)
              print('porTendencias: '+str(var3))

              #detectar numero de palabras
              conteoTextoCorto=0
              for h in conteoTendencias(HashtagList)[1]:
                  if len([tweet.full_text for tweet in results][h].split())<10 and 'https://t.' in [tweet.full_text for tweet in results][h]:
                      conteoTextoCorto+=1
                      
              var4 =conteoTextoCorto/len(data_set) 
              porConteo.append(var4)
              print('PorConteo: '+str(var4))
              
              #Conteo de Fuentes de Tweets diferentes a Twitter, Android o iPhone
              words_for_matching = ['Twitter','Android','iPhone']
              match = [ False if any(xs in s for xs in words_for_matching) else True for s in data_set['source'].tolist()]
              match_sum = sum(match)
              var5 = match_sum/len(data_set)
              porSource.append(var5)
              if match_sum>0:
                  
                  print('porSource: '+str(var5))
                  print('Suma de Fuentes Sospechosas = '+str(match_sum))
                  
              sumaTotal.append(0.2*var1+0.2*var2+0.2*var3+0.2*var4+0.2*var5)
              
              usuarios.append(results[0].author.screen_name)

              #Revisa score del Botometer
              var_botometer = getscore(results[0].author.screen_name)
              boto.append(var_botometer)
              print('Score Botometer: ' +str(var_botometer))

              #Asigna a cada usuario 'Bot','Spammer' o 'No Bot' de acuerdo al criterio ponderado
              if (0.10*var1+0.20*var2+0.25*var3+0.25*var4+0.2*var5) >= 0.34:
                if var_botometer >=3.25:
                  respuesta = 'Bot'
                if (var_botometer>2.5) and (var_botometer<3.25):
                  respuesta = 'Spammer'
                  
              elif (0.10*var1+0.20*var2+0.25*var3+0.25*var4+0.2*var5) < 0.34:
                respuesta = 'No Bot'

          else:
            respuesta = 'No Bot'
      else:
        respuesta = 'No Bot'
      
      respuestaFinal.append(respuesta)
      print(str(round((i+1)/len(users)*100,1))+'%')
      
  data = pd.DataFrame([],columns = ['username','RT','Multi','Tendencia','Texto Corto','Source'])
  data['RT']=porRT
  data['Multi']=porMulti
  data['Tendencia']=porTendencias
  data['Texto Corto']=porConteo
  data['username'] = usuarios
  data['Source']=porSource
  data['respuesta']=sumaTotal
  data['Botometer'] = boto
  data.to_excel('/content/data_bots.xlsx',index =False)
  df = pd.DataFrame([],columns = ['users','respuesta'])
  df['users'] = users
  df['respuesta'] = respuestaFinal
  df.to_excel('/content/clasificacion_bots.xlsx',index = False)
  #Reporta Usuarios clasificados como 'Bot'
  if reportar == True:
    denunciar_bots(clasificacion_bots=df)



    

¿Quiere reportar a los usuarios clasificados como "BOT"? (Si/No): si
Conteo Usuarios Reales: 1%
Conteo Usuarios Reales: 3%
Conteo Usuarios Reales: 4%
Conteo Usuarios Reales: 6%
Conteo Usuarios Reales: 7%
Conteo Usuarios Reales: 8%
Conteo Usuarios Reales: 10%
Conteo Usuarios Reales: 11%
Conteo Usuarios Reales: 13%
Conteo Usuarios Reales: 14%
Conteo Usuarios Reales: 15%
Conteo Usuarios Reales: 17%
Conteo Usuarios Reales: 18%
Conteo Usuarios Reales: 20%
Conteo Usuarios Reales: 21%
User Not Found
Conteo Usuarios Reales: 24%
Conteo Usuarios Reales: 25%
User Not Found
Conteo Usuarios Reales: 28%
Conteo Usuarios Reales: 30%
Conteo Usuarios Reales: 31%
User Not Found
Conteo Usuarios Reales: 34%
Conteo Usuarios Reales: 35%
Conteo Usuarios Reales: 37%
User Not Found
Conteo Usuarios Reales: 39%
Conteo Usuarios Reales: 41%
Conteo Usuarios Reales: 42%
Conteo Usuarios Reales: 44%
Conteo Usuarios Reales: 45%
Conteo Usuarios Reales: 46%
Conteo Usuarios Reales: 48%
Conteo Usuarios Reales: 49%
Conteo Us

In [22]:
clasificacion_bots = pd.read_excel('/content/clasificacion_bots.xlsx')
clasificacion_bots

,users,respuesta
0,ValenciaNels0n,No Bot
1,FisicoImpuro,No Bot
2,SoloCaracteres8,No Bot
3,Col_Salada,Bot
4,reydormilon,Spammer
...,...,...
62,LinaAlv2203,No Bot
63,xicel_,No Bot
64,willian23201618,No Bot
65,SoySoloYoyYa,Bot


In [17]:
data_bots = pd.read_excel('/content/data_bots.xlsx')
data_bots.head(50)

,username,RT,Multi,Tendencia,Texto Corto,Source,respuesta,Botometer
0,SoloCaracteres8,0.640000,0.700000,0.835000,0.510000,0,0.537000,2.5
1,Col_Salada,0.135000,0.915000,0.405000,0.330000,0,0.357000,3.3
2,reydormilon,0.950000,0.725000,0.395000,0.120000,0,0.438000,3.1
3,Littleeapple_,0.250000,0.805556,0.138889,0.027778,0,0.244444,1.6
4,Johana86975691,0.320000,0.800000,0.260000,0.060000,0,0.288000,3.4
5,alexbts93,0.041667,0.583333,0.166667,0.041667,0,0.166667,3.8
6,misiaaigo,0.064935,0.883117,0.415584,0.246753,0,0.322078,3.6
7,Andrea11807604,0.583333,0.812500,0.458333,0.395833,0,0.450000,3.6
8,DazMarthae,0.995000,0.035000,0.000000,0.000000,0,0.206000,2.7
9,OKogarashi,0.885000,0.510000,0.200000,0.050000,0,0.329000,2.1


In [16]:
bots_denunciados = pd.read_excel('/content/bots_denunciados.xlsx')
bots_denunciados

,bots denunciados


In [23]:
clasificacion_bots[clasificacion_bots['respuesta']=='Bot']

,users,respuesta
3,Col_Salada,Bot
13,misiaaigo,Bot
15,Andrea11807604,Bot
23,TThebestgroup,Bot
50,AndrsCo37681527,Bot
53,LeydiDaza9,Bot
60,3423Bichita,Bot
61,BTSdude1,Bot
65,SoySoloYoyYa,Bot


In [24]:
clasificacion_bots[clasificacion_bots['respuesta']=='Spammer']

,users,respuesta
4,reydormilon,Spammer


In [7]:
clasificacion_bots.shape

(412, 2)

In [9]:
data_bots.shape

(259, 8)

In [10]:
data_bots

,username,RT,Multi,Tendencia,Texto Corto,Source,respuesta,Botometer
0,andr3ss3rrato,0.325,0.435000,0.010000,0.000,0.0,0.154000,0.1
1,Florangelicma,0.865,0.175000,0.000000,0.000,0.0,0.208000,1.0
2,ALBERANiBAL9005,0.965,0.190000,0.005000,0.000,0.0,0.232000,3.4
3,lilianabernal87,0.280,0.055000,0.000000,0.000,0.0,0.067000,0.6
4,CarlosMoralesSv,1.000,0.305000,0.000000,0.000,1.0,0.461000,4.2
...,...,...,...,...,...,...,...,...
254,reydereyes2020,0.325,0.275000,0.005000,0.000,0.0,0.121000,0.0
255,NormaMu32781291,1.000,0.169492,0.033898,0.000,0.0,0.240678,0.0
256,piolispm,1.000,0.210000,0.000000,0.000,0.0,0.242000,0.0
257,lunaDM33,0.545,0.660000,0.005000,0.000,0.0,0.242000,0.0


In [11]:
data_bots[data_bots['Tendencia']>0.5]

,username,RT,Multi,Tendencia,Texto Corto,Source,respuesta,Botometer
19,meche9920,0.015000,0.960000,0.935000,0.305000,0.0,0.443000,3.2
25,Annabel41782867,0.316667,0.811111,0.683333,0.244444,0.0,0.411111,2.3
37,miarods,0.925000,0.420000,0.710000,0.125000,0.0,0.436000,1.8
125,SofaHer84054524,0.143617,0.654255,0.558511,0.015957,0.0,0.274468,2.0
185,Poca_Plata__,0.765000,0.660000,0.690000,0.335000,0.0,0.490000,0.0
187,Sergio10261572,0.205000,0.815000,0.765000,0.430000,0.0,0.443000,0.0
